In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

import warnings
warnings.filterwarnings('ignore')

In [26]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tanis\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [3]:
yes = pd.read_csv("yes.csv")
yes = yes["sequence"].to_list()

no = pd.read_csv("no.csv")
no = no["sequence"].to_list()

neither = pd.read_csv("neither.csv")
neither = neither["sequence"].to_list()

In [12]:
# creating train and test partitions for sequences
train_yes = yes[:int(0.85 * len(yes))]
test_yes = yes[int(0.85 * len(yes)):]

train_no = no[:int(0.85 * len(no))]
test_no = no[int(0.85 * len(no)):]

train_neither = neither[:int(0.85 * len(neither))]
test_neither = neither[int(0.85 * len(neither)):]

train_x = train_yes + train_no + train_neither
test_x = test_yes + test_no + test_neither

In [13]:
# creating train and test partitions for labels
t_y = np.full(len(train_yes), 1)
test_y = np.full(len(test_yes), 1)

t_n = np.full((len(train_neither)), 0.5)
test_n = np.full((len(test_neither)), 0.5)

t_no = np.full((len(train_no)), 0)
test_no_ = np.full(len(test_no), 0)

train_y = np.concatenate((t_y, t_no, t_n), axis=0)
test_y = np.concatenate((test_y, test_no_, test_n), axis=0)

In [31]:
def process(sequence):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    
    # tokenize sequences
    tokens = word_tokenize(sequence)

    sequences_clean = []
    for word in tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            stem_word = stemmer.stem(word)  # stemming word
            sequences_clean.append(stem_word)

    return sequences_clean

In [32]:
def build_freqs(sequences, ys):
    """Build frequencies.
    Input:
        sequences: sequences
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0/0.5/1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, seq in zip(yslist, sequences):
        for word in process(seq):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [35]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 298
